## 3.5 Pandas处理缺失值
《Python数据分析》课程讲义pandas模块内容, 参考教材为《Python数据科学手册》  

---

现实工作中, 很多**数据集都会有数据缺失的现象**. 为此，本节将介绍一些处理缺失值的通用规则, Pandas 对缺失值的表现形式, 并演示 Pandas 自带的几个处理缺失值的工具的用法. 本节以及全书涉及的缺失值主要有三种形式：null、NaN 或 NA. 


### 3.5.1 选择处理缺失值的方法
在数据表或 `DataFrame` 中有很多识别缺失值的方法. 一般情况下可以分为两种：一种方法是通过一个覆盖全局的**掩码**表示缺失值, 另一种方法是用一个**标签值**(sentinel value)表示缺失值. 

- 在掩码方法中, 掩码可能是一个与原数组维度相同的完整布尔类型数组, 也可能是用一个比特(0 或 1)表示有缺失值的局部状态. 

- 在标签方法中, 标签值可能是具体的数据(例如用 -9999 表示缺失的整数), 也可能是些极少出现的形式. 另外, 标签值还可能是更全局的值, 比如用 `NaN`(不是一个数)表示缺失的浮点数, 它是 IEEE 浮点数规范中指定的特殊字符. 

使用这两种方法之前都需要先综合考量：使用单独的掩码数组会额外出现一个布尔类型数组, 从而增加存储与计算的负担；而标签值方法缩小了可以被表示为有效值的范围, 可能需要在 CPU 或 GPU 算术逻辑单元中增加额外的(往往也不是最优的)计算逻辑. 通常使用的 `NaN` 也不能表示所有数据类型. 

### 3.5.2 Pandas的缺失值
Pandas 里处理缺失值的方式延续了 NumPy 程序包的方式, **并没有为浮点数据类型提供内置的 `NA` 作为缺失值.**

> Pandas 原本也可以按照 R 语言采用的比特模式为每一种数据类型标注缺失值, 但是这种方法非常笨拙. R 语言包含 4 种基本数据类型, 而 NumPy 支持的类型**远超** 4 种. 例如, R 语言只有一种整数类型, 而 NumPy 支持**14**种基本的整数类型, 可以根据精度、符号、编码类型按需选择. 如果要为 NumPy 的每种数据类型都设置一个比特标注缺失值, 可能需要为不同类型的不同操作耗费大量的时间与精力, 其工作量几乎相当于创建一个新的 NumPy 程序包. 另外, 对于一些较小的数据类型(例如 8 位整型数据), 牺牲一个比特作为缺失值标注的掩码还会导致其数据范围缩小. 当然, NumPy 也是支持掩码数据的, 也就是说可以用一个布尔掩码数组为原数组标注"无缺失值"或"有缺失值". Pandas 也集成了这个功能, 但是在存储、计算和编码维护方面都需要耗费不必要的资源, 因此这种方式并不可取. 

综合考虑各种方法的优缺点, **Pandas 最终选择用标签方法表示缺失值**, 包括两种 Python 原有的缺失值：
- Python 的 `None` 对象. 
- 浮点数据类型的 `NaN` 值 (浮点数).

后面我们将会发现, 虽然这么做也会有一些副作用, 但是在实际运用中的效果还是不错的. 

#### 3.5.2.1 None：Python对象类型的缺失值
Pandas 可以使用的第一种缺失值标签是 `None` , 它是一个 Python 单体对象, 经常在代码中表示缺失值. 由于 `None` 是一个 Python 对象, 所以**不能作为任何 NumPy / Pandas 数值数组类型的缺失值**, 只能用于 `'object'` 数组类型(即由 Python 对象构成的数组)：

In [7]:
import numpy as np
import pandas as pd

vals2 = np.array([1, 2, 3, 4])
print(vals2.dtype)


vals1 = np.array([1, None, 3, 4])
print(vals1.dtype)

int32
object


这里 `dtype=object` 表示 NumPy 认为由于这个数组是 Python 对象构成的, 因此将其类型判断为 `object`. 虽然这种类型在某些情景中非常有用, 对数据的任何操作最终都会在 Python 层面完成, 但是在进行常见的快速操作时, **`object`类型比其他原生类型数组要消耗更多的资源**：

In [8]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
57.5 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
1.67 ms ± 85.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



使用`object`对象构成的数组：就意味着如果你对一个包含 `None` 的数组进行累计操作, 如 `sum()` 或者 `min()`, 那么通常会出现类型错误：

In [9]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

这就是说, 在 Python 中没有定义整数与 `None` 之间的加法运算. 

#### 3.5.2.2 NaN：数值类型的缺失值

另一种缺失值的标签是 `NaN`(全称 Not a Number, **不是一个数字**), 是一种按照 IEEE 浮点数标准设计、在任何系统中都兼容的特殊浮点数：

In [ ]:
vals2 = np.array([1, np.nan, 3, 4])
print(vals2.dtype)

float64


请注意, NumPy 会为这个数组**选择一个原生浮点类型**, 这意味着和之前的 `object` 类型数组不同, 这个数组会被编译成 C 代码从而实现**快速向量化操作**. 你可以把 `NaN` 看作是一个数据类病毒——它会将与它接触过的数据同化. 无论和 `NaN` 进行何种操作, 最终结果都是 `NaN`：

In [ ]:
print(1 + np.nan)

nan


In [ ]:
print(0 *  np.nan)

nan


虽然这些累计操作的结果定义是合理的(即不会抛出异常), 但是并非总是有效的：

In [ ]:
print(vals2.sum(), vals2.min(), vals2.max())

nan nan nan


NumPy 也提供了一些特殊的累计函数, 它们可以**忽略缺失值的影响**：

In [ ]:
print(np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2))

8.0 1.0 4.0


谨记, `NaN` 是一种特殊的浮点数, 不是整数、字符串以及其他数据类型. 

#### 3.5.2.3 Pandas中NaN与None的差异

虽然 `NaN` 与 `None` 各有各的用处, 但是 Pandas 把它们看成是可以等价交换的, 在适当的时候会将两者进行替换：

In [ ]:
print('------pandas会将None对象转化为nan-----------')
print(pd.Series([1, np.nan, 2, None]))

------pandas会将None对象转化为nan-----------
0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64


Pandas 会将没有标签值的数据类型自动转换为 `NAN`. 例如, 当我们将整型数组中的一个值设置为 `np.nan` 时, 这个值就会**强制转换成浮点数缺失值 `NAN`.**

In [ ]:
x = pd.Series(range(2), dtype=int)
print(x)

0    0
1    1
dtype: int32


In [ ]:
x[0] = None
print(x)

0    NaN
1    1.0
dtype: float64


请注意, 除了将整型数组的缺失值强制转换为浮点数, Pandas 还会自动将 `None` 转换为 `NaN`. Pandas 对 `NA` 缺失值进行强制转换的规则如表 3-2 所示. 

**表3-2：Pandas对不同类型缺失值的转换规则**

| 类型               | 缺失值转换规则       | `NA`标签值         |
| ------------------ | -------------------- | ------------------ |
| `floating` 浮点型  | 无变化               | `np.nan`           |
| `object` 对象类型  | 无变化               | `None` 或 `np.nan` |
| `integer` 整数类型 | 强制转换为 `float64` | `np.nan`           |
| `boolean` 布尔类型 | 强制转换为 `object`  | `None 或 np.nan`   |

需要注意的是, Pandas 中字符串类型的数据通常是用 `object` 类型存储的. 

### 3.5.3 处理缺失值
我们已经知道, Pandas 基本上把 `None` 和 `NaN` 看成是可以等价交换的缺失值形式. 为了完成这种交换过程, Pandas 提供了一些方法来发现、剔除、替换数据结构中的缺失值, 主要包括以下几种: 

1. `isnull()`:创建一个布尔类型的掩码标签缺失值.   

2. `notnull()`: 与 `isnull()` 操作相反.   

3. `dropna()`: 返回一个剔除缺失值的数据.   

4. `fillna()`: 返回一个填充了缺失值的数据副本. 


#### 3.5.3.1 发现缺失值

Pandas 数据结构有两种有效的方法可以发现缺失值：`isnull()` 和 `notnull()`. 每种方法都返回布尔类型的掩码数据, 例如：

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
print(data)
print('----------data.isnull()-------------')
print(data.isnull())
print('-----------data.notnull()------------')
print(data.notnull())

0        1
1      NaN
2    hello
3     None
dtype: object
----------data.isnull()-------------
0    False
1     True
2    False
3     True
dtype: bool
-----------data.notnull()------------
0     True
1    False
2     True
3    False
dtype: bool


就像在 3.3 节中介绍的, **布尔类型掩码数组**可以直接作为 `Series` 或 `DataFrame` 的掩码索引, 过滤缺失值：

In [ ]:
print(data[data.notnull()])

0        1
2    hello
dtype: object


在 `Series` 里使用的 `isnull()` 和 `notnull()` 同样适用于 `DataFrame`, 产生的结果同样是**二维的布尔类型数组**. 

#### 3.5.3.2 剔除缺失值
除了前面介绍的掩码方法, 还有两种很好用的缺失值处理方法, 分别是 `dropna()`(剔除缺失值)和 `fillna()`(填充缺失值). 在 `Series` 上使用这些方法非常简单：

In [ ]:
print(data.dropna())

0        1
2    hello
dtype: object


而在 `DataFrame` 上使用它们时需要设置一些参数, 例如下面的 `DataFrame`：

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


> 没法从 `DataFrame` 中**单独剔除一个值**, 要么是剔除缺失值所在的整行, 要么是整列. 根据实际需求, 有时你需要剔除整行, 有时可能是整列, `DataFrame` 中的 `dropna()` 会有一些参数可以配置. 

默认情况下, `dropna()` 会剔除**任何**包含缺失值的整行数据：

In [ ]:
df.dropna()

,0,1,2
1,2.0,3.0,5


可以设置按不同的坐标轴剔除缺失值, 比如 `axis=1`(或 `axis='columns'`)会剔除任何包含缺失值的整列数据：

In [ ]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


但是这么做也会把非缺失值一并剔除, 因为可能有时候只需要剔除**全部**是缺失值的行或列, 或者绝大多数是缺失值的行或列. 

>这些需求可以通过设置 `how` 或 `thresh` 参数来满足, 它们可以设置剔除行或列缺失值的数量阈值. 

默认设置是 `how='any'`, 也就是说只要有缺失值就剔除整行或整列(通过 axis 设置坐标轴). 你还可以设置 `how='all'`, 这样就只会剔除全部是缺失值的行或列了：

In [ ]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [ ]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


还可以通过 `thresh` 参数设置行或列中**非缺失值**的最小数量, 从而实现更加个性化的配置：

In [ ]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


第 1 行与第 3 行被剔除了, 因为它们只包含两个非缺失值. 

#### 3.5.3.3 填充缺失值
有时候你可能并不想移除缺失值, 而是想把它们替换成有效的数值. 有效的值可能是像 0、1、2 那样单独的值, 也可能是经过填充(imputation)或转换(interpolation)得到的. 虽然你可以通过 `isnull()` 方法建立掩码来填充缺失值, 但是 Pandas 为此专门提供了一个 `fillna()` 方法, 它将返回填充了缺失值后的数组副本. 

来用下面的 `Series` 演示：

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
print(data)

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64


我们将用一个单独的值来填充缺失值, 例如用 0：

In [ ]:
print(data.fillna(0))

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64


可以用缺失值前面的有效值来从前往后填充(forward-fill)：

In [ ]:
# 从前往后填充
print(data.fillna(method='ffill'))

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64


也可以用缺失值后面的有效值来从后往前填充(back-fill)：

In [ ]:
# 从后往前填充
print(data.fillna(method='bfill'))

NameError: name 'data' is not defined

`DataFrame` 的操作方法与 `Series` 类似, 只是在填充时需要设置坐标轴参数 `axis`：

In [ ]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [ ]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


需要注意的是, 假如在从前往后填充时, 需要填充的缺失值前面没有值, 那么它就仍然是缺失值. 